In [1]:
import pandas as pd
import os, sys
import nltk
sys.path.append(".")
import preprocessing.cleaning_functions
import datetime
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [3]:
cwd = os.getcwd()
data_path = os.path.join(cwd, "data")

news_df = pd.read_csv(os.path.join(data_path, "RedditNews.csv"))
stock_df = pd.read_csv(os.path.join(data_path, "upload_DJIA_table.csv"))

In [4]:
df = preprocessing.cleaning_functions.clean_and_combine(stock_df, news_df)

The labels correspond to a change in stock value as follows:

0. change < -2%
1. -2% < change < 0%
2. 0% < change < 2%
3. 2% < change

In [5]:
train = df[df["Date"] < datetime.datetime(2015, 1, 1)]
test = df[df["Date"] > datetime.datetime(2014, 12, 31)]

In [6]:
trainheadlines = []
for row in range(0,len(train.index)):
    trainheadlines.append(' '.join(str(x) for x in train.iloc[row, 3]))

In [7]:
basicvectorizer = CountVectorizer()
basictrain = basicvectorizer.fit_transform(trainheadlines)
print(basictrain.shape)

(1611, 31674)


In [8]:
advancedvectorizer = TfidfVectorizer( min_df=0.1, max_df=0.7, max_features = 200000, ngram_range = (1, 1))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)
print(advancedtrain.shape)

(1611, 529)


In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

advancedmodel = MultinomialNB(alpha=0.01)
advancedmodel = advancedmodel.fit(advancedtrain, train["change_cat"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds4 = advancedmodel.predict(advancedtest)
acc4=accuracy_score(test['change_cat'], preds4)

In [10]:
print('NBayes 1 accuracy: ', acc4)

NBayes 1 accuracy:  0.5079365079365079
